In [2]:
import gc
import os

import json
import numpy as np
import pandas as pd
import random
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset

import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint

from transformers import AutoTokenizer, AutoModel, DebertaV2Config, DebertaV2Model

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TRANSFORMERS_CACHE'] = './cache/'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
import re

def text_preprocess(text):
    text = re.sub("[“”«»″]", '"', text)
    text = re.sub("‘’", '"', text)
    text = re.sub("’", "'", text)
    text = re.sub("[—–]+", '-', text)
    text = re.sub("【", '(', text)
    text = re.sub("】", ')', text)
    text = re.sub("[\\\]+|/", ' ', text)
    text = re.sub("[\n\t\xa0]", ' ', text)
    text = re.sub("[\u200b\xad•]", '', text)
    text = re.sub('"+', '"', text)
    text = re.sub(" +", ' ', text)
    text = text.strip()
    return text

In [4]:
torch.__version__, torch.cuda.is_available()

('2.0.1+cu117', True)

In [5]:
pl.seed_everything(56, workers=True)

56

In [6]:
train_data = pd.read_parquet('./datasets/train_data.parquet').set_index('variantid')
train_data['categories'] = train_data['categories'].apply(lambda x: json.loads(x))
train_data['main_pic_embeddings_resnet_v1'] = train_data['main_pic_embeddings_resnet_v1'].apply(lambda x: x[0])
train_data

,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64,characteristic_attributes_mapping
variantid,,,,,,,
51195767,"Удлинитель Партнер-Электро ПВС 2х0,75 ГОСТ,6A,...","{'1': 'EPG', '2': 'Электроника', '3': 'Сетевые...",[оранжевый],None,"[0.04603629, 0.18839523, -0.09973055, -0.66368...","[-0.47045058, 0.67237014, 0.48984158, -0.54485...","{""Номинальный ток, А"":[""10""],""Цвет товара"":[""о..."
53565809,Магнитный кабель USB 2.0 A (m) - USB Type-C (m...,"{'1': 'EPG', '2': 'Электроника', '3': 'Кабели ...",[красный],"[[0.26863545, -0.3130674, 0.29023397, 0.073978...","[1.1471839, -0.665361, 0.7745614, 0.26716197, ...","[-0.6575592, 0.6522429, 0.5426037, -0.54347897...","{""Конструктивные особенности"":[""Магнитная конс..."
56763357,"Набор микропрепаратов Konus 25: ""Клетки и ткан...","{'1': 'EPG', '2': 'Электроника', '3': 'Оптичес...",None,"[[0.66954195, 1.0643557, 0.78324044, -0.338267...","[-0.90570974, 1.0296293, 1.0769907, 0.27746, -...","[-0.7384308, 0.70784587, 0.3012653, -0.3583719...","{""Тип аксессуара"":[""Набор микропрепаратов""],""Б..."
56961772,"Мобильный телефон BQ 1848 Step, черный","{'1': 'EPG', '2': 'Электроника', '3': 'Смартфо...",[черный],"[[0.6580482, -0.35763323, -0.16939065, -0.4249...","[0.13133773, -0.5577079, 0.32498044, 0.1917174...","[-0.44812852, 0.5283565, 0.28981736, -0.506841...","{""Тип карты памяти"":[""microSD""],""Число SIM-кар..."
61054740,"Штатив трипод Tripod 330A для фотоаппаратов, в...","{'1': 'EPG', '2': 'Электроника', '3': 'Штативы...",[черный],"[[-0.10406649, 0.080646515, -0.28668788, 0.739...","[0.21696381, 0.10989461, -0.08012986, 0.691861...","[-0.72692573, 0.75206333, 0.37740713, -0.52502...","{""Материал"":[""Металл""],""Количество секций, шт""..."
...,...,...,...,...,...,...,...
820128810,"Комплект 2 шт, Чернила Cactus CS-EPT6733B пурп...","{'1': 'EPG', '2': 'Электроника', '3': 'Расходн...",[пурпурный],None,"[-1.4492652, -0.80129164, -0.12344764, 0.71945...","[-0.8253241, 0.6785133, 0.53978086, -0.4888316...","{""Тип"":[""Чернила для принтера""],""Бренд печатаю..."
821135769,"Защитное стекло закаленное Xiaomi Redmi 7, Y3 ...","{'1': 'EPG', '2': 'Электроника', '3': 'Защитны...",[черный],"[[0.09564891, 0.27437285, -0.19054827, -0.7992...","[0.012127608, -0.8534423, 0.5415518, -0.449125...","[-0.7413257, 0.46105132, 0.5639801, -0.5462132...","{""Вид стекла"":[""3D""],""Тип"":[""Защитное стекло""]..."
822095690,Системный блок ЮКОМС 9400-268 (AMD A6-9400 (3....,"{'1': 'EPG', '2': 'Электроника', '3': 'Компьют...",[черный],None,"[0.4248176, -0.15944786, -0.22844064, 0.427686...","[-0.49261805, 0.56726897, 0.7037877, -0.697246...","{""Общий объем HDD, ГБ"":[""10000""],""Видеокарта"":..."


In [7]:
test_data = pd.read_parquet('./datasets/test_data.parquet').set_index('variantid')
test_data['categories'] = test_data['categories'].apply(lambda x: json.loads(x))
test_data['main_pic_embeddings_resnet_v1'] = test_data['main_pic_embeddings_resnet_v1'].apply(lambda x: x[0])
test_data

,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64,characteristic_attributes_mapping
variantid,,,,,,,
51201254,Колодка TDM Electric четырехместная без заземл...,"{'1': 'EPG', '2': 'Электроника', '3': 'Сетевые...",[белый],"[[0.34383398, -0.2962618, 0.07987049, -0.08257...","[0.38310742, -0.7876679, 0.5018278, 0.20900711...","[-0.5060825, 0.5773388, 0.59435517, -0.4958292...","{""Страна-изготовитель"":[""Китай""],""Бренд"":[""TDM..."
77151532,Клавиатура черная с черной рамкой для 25-011879,"{'1': 'EPG', '2': 'Электроника', '3': 'Запчаст...",[черный],None,"[0.50964713, 0.7958329, -1.4113188, 0.19993813...","[-0.43467724, 0.6614495, 0.48050267, -0.588880...","{""Страна-изготовитель"":[""Китай""],""Комплектация..."
89664856,"15.6"" Игровой ноутбук Acer Predator Helios 300...","{'1': 'EPG', '2': 'Электроника', '3': 'Компьют...",[черный],"[[0.7804302, -0.245446, -0.67754817, -0.614691...","[0.9958085, -0.113175124, -0.7623152, -0.91648...","[-0.70010763, 0.48152006, 0.47597092, -0.51727...","{""Видеокарта"":[""NVIDIA GeForce RTX 2070 (8 Гб)..."
90701982,Портативная колонка Borofone BR7 Empyreal Spor...,"{'1': 'EPG', '2': 'Электроника', '3': 'Акустик...","[red, красный]","[[-0.24636984, -1.0719914, -0.49986655, 0.3423...","[-0.26596686, -1.143009, -0.5289628, 0.4285588...","[-0.73135185, -0.039796613, 0.38907066, -0.496...","{""Основной материал корпуса"":[""Металл""],""Макси..."
92484118,Аккумулятор для Meizu BA712 ( M6s ),"{'1': 'EPG', '2': 'Электроника', '3': 'Батарей...",None,None,"[0.42047608, 0.75828516, 0.5440093, -0.0068945...","[-0.600158, 0.13944691, 0.48706242, -0.5050975...","{""Рекомендовано для"":[""Meizu""],""Бренд"":[""Meizu..."
...,...,...,...,...,...,...,...
702785891,Кабель USB - Lightning HOCO X21 PLUS (черно-бе...,"{'1': 'EPG', '2': 'Электроника', '3': 'Кабели ...",[черный],"[[1.1820095, -0.16312826, 1.4916217, 0.0288323...","[0.3297959, -0.16444838, 0.9350716, 0.34787956...","[-0.66597974, 0.7140731, 0.43572947, -0.445908...","{""Бренд"":[""hoco""],""Тип"":[""Кабель""],""Цвет товар..."
704096517,Блок питания для ноутбука Asus f5gl (19V 90W 4...,"{'1': 'EPG', '2': 'Электроника', '3': 'Зарядны...",[черный],"[[-0.013610864, -0.68512607, 0.77639246, -1.04...","[0.2785852, -0.16053033, 1.1653559, 1.0619084,...","[-0.7575411, 0.4196694, 0.46428213, -0.4916808...","{""Комплектация"":[""Зарядное устройство, сетевой..."
705874953,Оперативная память HyperX FURY Black DDR4 2666...,"{'1': 'EPG', '2': 'Электроника', '3': 'Операти...",[black],"[[0.34073856, 0.65070343, 0.31146732, 1.261663...","[0.31382418, 0.60041714, 0.3067428, 1.1233345,...","[-0.60506856, 0.4477128, 0.62255704, -0.720129...","{""Тайминги"":[""16-18-18-29""],""Пропускная способ..."


In [8]:
train_pairs = pd.read_parquet('./datasets/train_pairs_w_target.parquet')
train_pairs['target'] = train_pairs['target'].astype(int)
train_pairs

,target,variantid1,variantid2
0,0,51197862,51198054
1,1,53062686,536165289
2,1,53602615,587809782
3,1,53888651,89598677
4,0,56930698,551526166
...,...,...,...
306535,0,817327230,822083612
306536,0,817560551,818069912
306537,0,817854719,817857267
306538,0,820036017,820037019


In [9]:
test_pairs = pd.read_parquet('./datasets/test_pairs_wo_target.parquet')
test_pairs

,variantid1,variantid2
0,52076340,290590137
1,64525522,204128919
2,77243372,479860557
3,86065820,540678372
4,91566575,258840506
...,...,...
18079,666998614,667074522
18080,670036240,670048449
18081,670284509,684323809
18082,692172005,704805270


In [10]:
val_pairs = train_pairs[pd.read_csv('./datasets/val_idx.csv', index_col=0).values].copy()
train_pairs = train_pairs[pd.read_csv('./datasets/train_idx.csv', index_col=0).values].copy()

In [11]:
train_cat3 = set()
for categories in train_data.categories:
    train_cat3.add(categories['3'])
test_cat3 = set()
for categories in test_data.categories:
    test_cat3.add(categories['3'])
both_cat3 = train_cat3 & test_cat3
len(train_cat3), len(test_cat3), len(both_cat3)

(127, 74, 74)

In [12]:
both_cat3.add('rest')
categories_map = {}
for v in both_cat3:
    categories_map[v] = len(categories_map)

In [13]:
colors_mapper = {
 'ярко-синий': 'ярко-синий',
 'ярко-розовый': 'ярко-розовый',
 'ярко-зеленый': 'ярко-зеленый',
 'ярко-желтый': 'ярко-желтый',
 'янтарный': 'янтарный',
 'электрик': 'электрик',
 'экрю': 'экрю',
 'шоколадный': 'шоколадный',
 'черный': 'черный',
 'черно-синий': 'черно-синий',
 'черно-серый': 'черно-серый',
 'черно-красный': 'черно-красный',
 'черно-зеленый': 'черно-зеленый',
 'черн': 'черный',
 'чер': 'черный',
 'циан': 'бирюзовый',
 'цементный': 'цементный',
 'хаки': 'хаки',
 'фуксия': 'фуксия',
 'фисташковый': 'фисташковый',
 'фиолетовый': 'фиолетовый',
 'фиолетово-синий': 'фиолетово-синий',
 'фиолет': 'фиолетовый',
 'фиол': 'фиолетовый',
 'фиалковый': 'фиалковый',
 'тыквенный': 'тыквенный',
 'тыква': 'тыквенный',
 'травяной': 'травяной',
 'томатный': 'томатный',
 'тиффани': 'тиффани',
 'терракотовый': 'терракотовый',
 'терракота': 'терракотовый',
 'темно-фиолетовый': 'темно-фиолетовый',
 'темно-синий': 'темно-синий',
 'темно-серый': 'темно-серый',
 'темно-розовый': 'темно-розовый',
 'темно-оранжевый': 'темно-оранжевый',
 'темно-оливковый': 'темно-оливковый',
 'темно-красный': 'темно-красный',
 'темно-коричневый': 'темно-коричневый',
 'темно-зеленый': 'темно-зеленый',
 'темно-голубой': 'темно-голубой',
 'темно-бирюзовый': 'темно-бирюзовый',
 'темно-бежевый': 'темно-бежевый',
 'сливовый': 'сливовый',
 'сиреневый': 'сиреневый',
 'синий': 'синий',
 'сине-зеленый': 'сине-зеленый',
 'син': 'синий',
 'серый': 'серый',
 'серовато-зеленый': 'серовато-зеленый',
 'серо-коричневый': 'серо-коричневый',
 'серо-зеленый': 'серо-зеленый',
 'серо-голубой': 'серо-голубой',
 'серо-бежевый': 'серо-бежевый',
 'серебряный': 'серебряный',
 'серебристый': 'серебристый',
 'серебристо-серый': 'серебристо-серый',
 'сер': 'серый',
 'сепия': 'сепия',
 'светло-фиолетовый': 'светло-фиолетовый',
 'светло-синий': 'светло-синий',
 'светло-серый': 'светло-серый',
 'светло-розовый': 'светло-розовый',
 'светло-пурпурный': 'светло-пурпурный',
 'светло-коричневый': 'светло-коричневый',
 'светло-золотистый': 'светло-золотистый',
 'светло-зеленый': 'светло-зеленый',
 'светло-желтый': 'светло-желтый',
 'светло-голубой': 'светло-голубой',
 'светло-бирюзовый': 'светло-бирюзовый',
 'светло-бежевый': 'светло-бежевый',
 'сапфировый': 'сапфировый',
 'салатовый': 'салатовый',
 'рыжий': 'рыжий',
 'розовый': 'розовый',
 'розово-фиолетовый': 'розово-фиолетовый',
 'розово-золотой': 'розово-золотой',
 'разноцветный': 'разноцветный',
 'пурпурный': 'пурпурный',
 'пурпурно-фиолетовый': 'пурпурно-фиолетовый',
 'песочный': 'песочный',
 'перу': 'перу',
 'персиковый': 'персиковый',
 'охра': 'охра',
 'орхидея': 'орхидея',
 'оранжевый': 'оранжевый',
 'оранжево-розовый': 'оранжево-розовый',
 'оливковый': 'оливковый',
 'огненно-красный': 'огненно-красный',
 'нефритовый': 'нефритовый',
 'небесный': 'небесный',
 'мятный': 'мятный',
 'мятно-зеленый': 'мятно-зеленый',
 'мята': 'мятный',
 'мультиколор': 'мультиколор',
 'морковный': 'морковный',
 'молочный': 'молочный',
 'многоцветный': 'многоцветный',
 'медный': 'медный',
 'марсала': 'марсала',
 'малиновый': 'малиновый',
 'малиново-красный': 'малиново-красный',
 'малахитовый': 'малахитовый',
 'льняной': 'льняной',
 'лимонный': 'лимонный',
 'лиловый': 'лиловый',
 'латунный': 'латунный',
 'лаймовый': 'лаймовый',
 'лайм': 'лаймовый',
 'лазурный': 'лазурный',
 'лавандовый': 'лавандовый',
 'лаванда': 'лавандовый',
 'кремовый': 'кремовый',
 'красный': 'красный',
 'красновато-коричневый': 'красновато-коричневый',
 'красно-оранжевый': 'красно-оранжевый',
 'красно-коричневый': 'красно-коричневый',
 'красн': 'красный',
 'крас': 'красный',
 'кофейный': 'кофейный',
 'космос': 'космос',
 'коричневый': 'коричневый',
 'коричнево-красный': 'коричнево-красный',
 'коричнево-бежевый': 'коричнево-бежевый',
 'коралловый': 'коралловый',
 'кораллово-красный': 'кораллово-красный',
 'кобальтовый': 'кобальтовый',
 'кирпичный': 'кирпичный',
 'кирпично-красный': 'кирпично-красный',
 'кварцевый': 'кварцевый',
 'кардинал': 'кардинал',
 'канареечный': 'канареечный',
 'камуфляжный': 'камуфляжный',
 'индиго': 'индиго',
 'изумрудный': 'изумрудный',
 'изумрудно-зеленый': 'изумрудно-зеленый',
 'изумруд': 'изумрудный',
 'золотой': 'золотой',
 'золотистый': 'золотистый',
 'зеленый': 'зеленый',
 'зелено-серый': 'зелено-серый',
 'зел': 'зеленый',
 'жемчужно-белый': 'жемчужно-белый',
 'желтый': 'желтый',
 'желто-розовый': 'желто-розовый',
 'желто-зеленый': 'желто-зеленый',
 'желт': 'желтый',
 'гусеница': 'гусеница',
 'грушевый': 'грушевый',
 'графит': 'графит',
 'гранитный': 'гранитный',
 'гранатовый': 'гранатовый',
 'горчичный': 'горчичный',
 'голубой': 'голубой',
 'голуб': 'голубой',
 'глициния': 'глициния',
 'вишня': 'вишневый',
 'вишневый': 'вишневый',
 'васильковый': 'васильковый',
 'ванильный': 'ванильный',
 'бурый': 'бурый',
 'бронзовый': 'бронзовый',
 'бордовый': 'бордовый',
 'бордо': 'бордовый',
 'болотный': 'болотный',
 'бледно-розовый': 'бледно-розовый',
 'бледно-пурпурный': 'бледно-пурпурный',
 'бледно-желтый': 'бледно-желтый',
 'бирюзовый': 'бирюзовый',
 'бирюзово-зеленый': 'бирюзово-зеленый',
 'белый': 'белый',
 'белоснежный': 'белоснежный',
 'бело-зеленый': 'бело-зеленый',
 'бел': 'белый',
 'бежевый': 'бежевый',
 'бежево-серый': 'бежево-серый',
 'бежево-розовый': 'бежево-розовый',
 'баклажановый': 'баклажановый',
 'антрацитовый': 'антрацитовый',
 'аметистовый': 'аметистовый',
 'алый': 'алый',
 'аквамариновый': 'аквамариновый',
 'аква': 'аква',
 'абрикосовый': 'абрикосовый',
 'yellow': 'желтый',
 'wine': 'wine',
 'white': 'белый',
 'violet': 'фиолетовый',
 'vanilla': 'ванильный',
 'ultramarine': 'ultramarine',
 'turquoise': 'бирюзовый',
 'tomato': 'томатный',
 'teal': 'teal',
 'tan': 'tan',
 'snow': 'snow',
 'silver': 'серебряный',
 'sapphire': 'сапфировый',
 'red': 'красный',
 'purple': 'фиолетовый',
 'pink': 'розовый',
 'peru': 'перу',
 'pear': 'грушевый',
 'peach': 'персиковый',
 'orchid': 'орхидея',
 'orange': 'оранжевый',
 'olive': 'оливковый',
 'navy': 'navy',
 'magenta': 'пурпурный',
 'linen': 'linen',
 'lime': 'лаймовый',
 'lilac': 'сиреневый',
 'lemon': 'lemon',
 'lavender': 'лавандовый',
 'khaki': 'хаки',
 'jade': 'нефритовый',
 'ivory': 'ivory',
 'indigo': 'индиго',
 'grey': 'серый',
 'green': 'зеленый',
 'gray': 'серый',
 'gold': 'золотой',
 'fuchsia': 'фуксия',
 'flax': 'flax',
 'emerald': 'emerald',
 'denim': 'denim',
 'cyan': 'бирюзовый',
 'cream': 'кремовый',
 'corn': 'corn',
 'coral': 'коралловый',
 'copper': 'медный',
 'cobalt': 'кобальтовый',
 'chocolate': 'шоколадный',
 'burgundy': 'бордовый',
 'buff': 'buff',
 'brown': 'коричневый',
 'bronze': 'бронзовый',
 'brass': 'латунный',
 'blue': 'голубой',
 'blond': 'blond',
 'black': 'черный',
 'beige': 'бежевый',
 'azure': 'лазурный',
 'aquamarine': 'аквамариновый',
 'aqua': 'аквамариновый',
 'amethyst': 'аметистовый',
 'amber': 'янтарный'
}

In [14]:
color_vocab = {}
for color, v in colors_mapper.items():
    color_vocab[v] = len(color_vocab) + 1

In [15]:
class Args:
    batch_size = 96
    epochs = 10
    lr = 1e-5
    
args = Args()

In [28]:
class ItemsDataset(Dataset):
    def __init__(self, pairs, data):
        super().__init__()
        self.pairs = pairs.values
        self.pairs_len = len(self.pairs)
        
        self.main_pic_embs = data['main_pic_embeddings_resnet_v1']
        
        categories = data['categories'].copy().apply(lambda x: x['3'])
        categories[~categories.isin(categories_map)] = 'rest'
        self.categories = categories.apply(lambda v: categories_map[v])
        
        def color_to_idx(colors):
            if colors is None:
                return []
            return [color_vocab[colors_mapper[color]] for color in colors]
        def drop_dup_colors(colors):
            if colors is None:
                return []
            res = []
            for v in colors:
                if v not in res:
                    res.append(v)
            return res
        colors = data['color_parsed'].copy().apply(color_to_idx).apply(drop_dup_colors)
        def pad_colors(colors):
            max_colors = 17
            if len(colors) > max_colors:
                return colors[:max_colors]
            return colors + [0] * (max_colors - len(colors))
        self.colors = colors.apply(pad_colors)
            
        self.names = data['name'].apply(text_preprocess)
        
        self.name_bert_embs = data['name_bert_64']
        
    def __len__(self):
        return self.pairs_len

    def __getitem__(self, idx):
        target, id1, id2 = self.pairs[idx, :]
        return (
            self.categories[id1],
            torch.tensor(self.colors[id1]),
            self.names[id1],
            torch.tensor(self.main_pic_embs[id1]),
            torch.tensor(self.name_bert_embs[id1]),
            
            self.categories[id2],
            torch.tensor(self.colors[id2]),
            self.names[id2],
            self.main_pic_embs[id2],
            torch.tensor(self.name_bert_embs[id2]),
            
            target
        )

In [29]:
def get_data_loader(pairs, data, batch_size, drop_last, shuffle):
    dataset = ItemsDataset(pairs, data)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=35,
        drop_last=drop_last,
        shuffle=shuffle,
        pin_memory=True
    )
    return data_loader

In [30]:
def get_loaders(args):
    train_loader = get_data_loader(
        pairs=train_pairs,
        data=train_data,
        batch_size=args.batch_size,
        drop_last=True,
        shuffle=True
    )
    
    val_loader = get_data_loader(
        pairs=val_pairs,
        data=train_data,
        batch_size=args.batch_size,
        drop_last=False,
        shuffle=False
    )
    return train_loader, val_loader

In [31]:
train_loader, val_loader = get_loaders(args)
len(train_loader), len(val_loader)

(2128, 1065)

In [ ]:
next(iter(val_loader))

In [37]:
class MultiModalNet(pl.LightningModule):
    margin = 0.75
    
    def __init__(self):
        super(MultiModalNet, self).__init__()
        
        # attrs
        self.category_embedding = nn.Embedding(
            num_embeddings=len(categories_map), 
            embedding_dim=len(categories_map) // 2, 
            padding_idx=None
        )
        
        self.color_embedding = nn.Embedding(
            num_embeddings=len(color_vocab) + 2, 
            embedding_dim=(len(color_vocab) + 2) // 2, 
            padding_idx=0
        )
        self.color_lstm_hidden = 64
        self.color_lstm = nn.LSTM(
            input_size=(len(color_vocab) + 2) // 2, 
            hidden_size=self.color_lstm_hidden, 
            num_layers=1, 
            batch_first=True,
            bidirectional=True
        )
        
        # name
        self.LaBSE_tokenizer = AutoTokenizer.from_pretrained('cointegrated/LaBSE-en-ru')
        self.LaBSE_model = AutoModel.from_pretrained('cointegrated/LaBSE-en-ru')
        self.LaBSE_fc = nn.Linear(768, 768)
        
        # net
        input_size = len(categories_map) // 2 + 2*self.color_lstm_hidden + 768 + 128 + 64
        output_size = 768
        self.bn = nn.BatchNorm1d(input_size)
        self.embedding_dropout = nn.Dropout(p=0.05)
            
        deberta_cfg = DebertaV2Config(
            hidden_size=input_size,
            num_hidden_layers=1,
            num_attention_heads=1,
            intermediate_size=1024,
        )
        self.deberta = DebertaV2Model(deberta_cfg, ).encoder
        
        features_num = 2 * input_size
        embedding_size = (features_num + output_size) // 2
        self.neck = nn.Sequential(
            nn.BatchNorm1d(features_num),
            nn.Linear(features_num, embedding_size, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(embedding_size),
            nn.Linear(embedding_size, embedding_size, bias=False),
            nn.BatchNorm1d(embedding_size),
        )
        
        self.output_layer = nn.Linear(embedding_size, output_size)
        
    def forward(self, categories, colors, names, pic_embs, name_bert_embs):
        categories_output = self.category_embedding(categories)
        
        colors_emb = self.color_embedding(colors)
        output, (ht, ct) = self.color_lstm(colors_emb)
        out_forward = output[:, -1, :self.color_lstm_hidden]
        out_reverse = output[:, 0, self.color_lstm_hidden:]
        colors_output = torch.cat([out_forward, out_reverse], 1)
        
        encoded_input = self.LaBSE_tokenizer(
            names, padding=True, truncation=True, max_length=256, return_tensors='pt'
        ).to('cuda')
        model_output = self.LaBSE_model(**encoded_input)
        embeddings = torch.nn.functional.normalize(model_output.pooler_output)    
        names_output = self.LaBSE_fc(embeddings)
        
        pics_output = torch.nn.functional.normalize(pic_embs)
        
        names_bert_output = torch.nn.functional.normalize(name_bert_embs)
        
        x = torch.cat([categories_output, colors_output, names_output, pics_output, names_bert_output], dim=1)
        x = self.bn(x)
        x = self.embedding_dropout(x)
        x = x.unsqueeze(1)
        attention_mask = torch.ones((x.shape[0], 1), device='cuda')
        last_hidden = self.deberta(x, attention_mask)
        last_hidden = torch.concat([last_hidden[0].mean(1), last_hidden[0].max(1)[0]], -1)
        outputs = self.neck(last_hidden)
        outputs = self.output_layer(outputs)
        outputs = torch.nn.functional.normalize(outputs)    
        return outputs

    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=args.lr, betas=(0.9, 0.999), weight_decay=0.05
        )
        return optimizer
    
    def training_step(self, batch, batch_idx):
        # self.log('step', batch_idx, logger=True, on_epoch=True)
        categories1, colors1, names1, pic_embs1, name_bert_embs1,\
        categories2, colors2, names2, pic_embs2, name_bert_embs2,\
        labels = batch
        out1 = self.forward(categories1, colors1, names1, pic_embs1, name_bert_embs1)
        out2 = self.forward(categories2, colors2, names2, pic_embs2, name_bert_embs2)
        
        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("train_loss", loss, on_step=False, logger=False, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):        
        categories1, colors1, names1, pic_embs1, name_bert_embs1,\
        categories2, colors2, names2, pic_embs2, name_bert_embs2,\
        labels = batch
        out1 = self.forward(categories1, colors1, names1, pic_embs1, name_bert_embs1)
        out2 = self.forward(categories2, colors2, names2, pic_embs2, name_bert_embs2)
        
        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("val_loss", loss, logger=False, on_epoch=True, prog_bar=True)   
        
        try:
            auc = roc_auc_score(labels.detach().cpu(), 1 - dists.detach().cpu())
        except:
            auc = 0
            
        self.log("val_auc", auc, logger=False, on_epoch=True, prog_bar=True)
        
    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader
    
    def predict_step(self, batch, batch_idx):
        categories1, colors1, names1, pic_embs1, name_bert_embs1,\
        categories2, colors2, names2, pic_embs2, name_bert_embs2,\
        labels = batch
        out1 = self.forward(categories1, colors1, names1, pic_embs1, name_bert_embs1)
        out2 = self.forward(categories2, colors2, names2, pic_embs2, name_bert_embs2)
        
        dists = nn.PairwiseDistance()(out1, out2)
        return torch.cat([out1, out2, (1 - dists).unsqueeze(-1)], dim=1).detach().cpu()

In [38]:
model = MultiModalNet()
model

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MultiModalNet(
  (category_embedding): Embedding(75, 37)
  (color_embedding): Embedding(181, 90, padding_idx=0)
  (color_lstm): LSTM(90, 64, batch_first=True, bidirectional=True)
  (LaBSE_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(55083, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (o

In [39]:
import torchinfo

torchinfo.summary(model)

Layer (type:depth-idx)                                       Param #
MultiModalNet                                                --
├─Embedding: 1-1                                             2,775
├─Embedding: 1-2                                             16,290
├─LSTM: 1-3                                                  79,872
├─BertModel: 1-4                                             --
│    └─BertEmbeddings: 2-1                                   --
│    │    └─Embedding: 3-1                                   42,303,744
│    │    └─Embedding: 3-2                                   393,216
│    │    └─Embedding: 3-3                                   1,536
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─BertEncoder: 2-2                                      --
│    │    └─ModuleList: 3-6                                  85,054,464
│    └─BertPooler: 2-3                                       

In [ ]:
checkpoint_cb = ModelCheckpoint(
    dirpath='./MultiModal/', filename='products-{epoch:02d}-{val_auc:.4f}-normalize', monitor='val_auc', mode='max'
)

trainer = pl.Trainer(
    logger=False, # CSVLogger('./'),
    enable_checkpointing=True,
    callbacks=[checkpoint_cb],
    accelerator='gpu', 
    devices=[0],
    profiler='advanced',
    precision="16-mixed",
    check_val_every_n_epoch=1,
    max_epochs=args.epochs
)

trainer.fit(model)

In [ ]:
test_preds = trainer.predict(model, val_loader)

In [24]:
roc_auc_score(val_pairs.target, np.concatenate([pred.numpy()[:, -1] for pred in test_preds]))

0.8869868830887131

# Инференс

In [25]:
train_dataset = ItemsDataset(train_pairs, train_data)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=args.batch_size,
    num_workers=37,
    drop_last=False,
    shuffle=False,
    pin_memory=True
)

In [ ]:
features = np.concatenate([pred.numpy() for pred in trainer.predict(model, train_loader)])

In [ ]:
val_features = np.concatenate([pred.numpy() for pred in trainer.predict(model, val_loader)])

In [28]:
from catboost import CatBoostClassifier, Pool, cv

In [29]:
train_pool = Pool(
    data=features,
    label=train_pairs.target,
)

val_pool = Pool(
    data=val_features,
    label=val_pairs.target,
)

In [30]:
params = {
    'loss_function': 'Logloss',
    'custom_metric': ['AUC'],
    'task_type': 'CPU',
}

In [31]:
model_cb = CatBoostClassifier(**params, random_seed=56)
model_cb.fit(train_pool, eval_set=val_pool, verbose=False, plot=True, early_stopping_rounds=100)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [32]:
np.max(model_cb.get_evals_result()['validation']['AUC'])

0.908631905205787

In [33]:
cv_data = cv(
    params=params,
    pool=val_pool,
    fold_count=3,
    shuffle=True,
    partition_random_seed=0,
    plot=True,
    stratified=True,
    verbose=False
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/3]

bestTest = 0.3730102263
bestIteration = 998

Training on fold [1/3]

bestTest = 0.3740584124
bestIteration = 999

Training on fold [2/3]

bestTest = 0.3787170071
bestIteration = 999



In [34]:
max_iter = cv_data['test-AUC-mean'].argmax()
print(f"Best iteration: {max_iter}\nBest AUC: {round(cv_data.iloc[max_iter]['test-AUC-mean'], 4)}±{round(cv_data.iloc[max_iter]['test-AUC-std'], 4)}")

Best iteration: 999
Best AUC: 0.9095±0.0016


In [35]:
test_pairs['target'] = -1
test_pairs = test_pairs[['target', 'variantid1', 'variantid2']]
test_dataset = ItemsDataset(test_pairs, test_data)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=args.batch_size,
    num_workers=0,
    drop_last=False,
    shuffle=False,
    pin_memory=True
)

In [ ]:
test_features = np.concatenate([pred.numpy() for pred in trainer.predict(model, test_loader)])

In [37]:
train_embeds = pd.Series(index=train_data.index, dtype='object', name='multimodal_tuned_768')
train_embeds[val_pairs.variantid1] = list(val_features[:, :768])
train_embeds[val_pairs.variantid2] = list(val_features[:, 768:768*2])
train_embeds

variantid
51195767     [0.0023719287, 0.04152827, 0.0018715167, 0.035...
53565809     [0.022112135, -0.007468411, -0.0068747965, 0.0...
56763357                                                   NaN
56961772                                                   NaN
61054740     [0.01275732, 0.034349203, 0.008032634, 0.03303...
                                   ...                        
820128810                                                  NaN
821135769    [-0.015492215, -0.01956236, -0.008062369, -0.0...
822095690                                                  NaN
822101044                                                  NaN
822394794                                                  NaN
Name: labse_tuned_768, Length: 457063, dtype: object

In [38]:
test_embeds = pd.Series(index=test_data.index, dtype='object', name='multimodal_tuned_768')
test_embeds[test_pairs.variantid1] = list(test_features[:, :768])
test_embeds[test_pairs.variantid2] = list(test_features[:, 768:768*2])
test_embeds

variantid
51201254     [-0.0036340018, 0.05191699, -0.019618921, 0.03...
77151532     [0.020562124, -0.027739627, 0.003007357, 0.033...
89664856     [0.008961296, 0.0074943216, -0.0042574126, 0.0...
90701982     [0.0002899492, 0.00082552544, -0.0036707376, 0...
92484118     [0.0523341, -0.028439298, -0.0205805, 0.019437...
                                   ...                        
702785891    [0.015837612, -0.020126382, 0.01076397, 0.0433...
704096517    [-0.0046698838, -0.00089860754, -0.025120754, ...
705874953    [-0.012161249, 0.049054068, -0.02890747, 0.063...
706965102    [-0.021119665, 0.045512594, -0.011890485, 0.03...
707476739    [0.004665166, 0.03663141, 0.003135435, 0.04310...
Name: labse_tuned_768, Length: 35730, dtype: object

In [39]:
train_embeds.to_pickle('./datasets/multimodal_tuned_train.pickle')

In [40]:
test_embeds.to_pickle('./datasets/multimodal_tuned_test.pickle')